# Problem 1

This is a complete simulation transport example. Each aspect of the simulation process is kept to a minimum:
- We use an orthogonal 2D grid;
- We introduce the concept of domain decomposition ("partitioning");
- The domain is homogeneous (single material, uniform isotropic external source), vacuum boundary conditions apply;
- The cross sections are given in a text file (with our OpenSn format); we use only one energy group in this example;
- The angular quadrature (discretization in angle) is introduced;
- The Linear Boltzmann Solver (LBS) options are keep to a minimum.

In [1]:
import os
import sys

## Using this Notebook
Before running this example, make sure that the **Python module of OpenSn** was installed.

### Converting and Running this Notebook from the Terminal
To run this notebook from the terminal, simply type:

`jupyter nbconvert --to python --execute problem_one.ipynb`.

To run this notebook in parallel (for example, using 4 processes), simply type:

`mpiexec -n 4 jupyter nbconvert --to python --execute problem_1.ipynb`.

In [2]:
from mpi4py import MPI
size = MPI.COMM_WORLD.size
rank = MPI.COMM_WORLD.rank

if rank == 0:
    print(f"Running the first LBS example with {size} MPI processors.")

Running the first LBS example with 1 MPI processors.


## Import Requirements

Import required classes and functions from the Python interface of OpenSn. Make sure that the path
to PyOpenSn is appended to Python's PATH.

In [3]:
# assuming that the execute dir is the notebook dir
# this line is not necessary when PyOpenSn is installed using pip
# sys.path.append("../../../..")

from pyopensn.mesh import OrthogonalMeshGenerator, KBAGraphPartitioner
from pyopensn.xs import MultiGroupXS
from pyopensn.source import VolumetricSource
from pyopensn.aquad import GLCProductQuadrature3DXYZ
from pyopensn.solver import DiscreteOrdinatesProblem, SteadyStateSolver
from pyopensn.diffusion import DFEMDiffusionSolver, CFEMDiffusionSolver
from pyopensn.fieldfunc import FieldFunctionInterpolationVolume, FieldFunctionGridBased
from pyopensn.context import UseColor, Finalize
from pyopensn.logvol import SphereLogicalVolume, BooleanLogicalVolume, RPPLogicalVolume
from pyopensn.math import Vector3, ScalarSpatialMaterialFunction
import numpy as np
import math

OpenSn version 0.0.1
2025-05-10 11:34:35 Running OpenSn with 1 processes.



In [4]:
from scipy.special import exp2

##### Disable colorized output.

In [5]:
UseColor(False)

## Mesh
Here, we will use the in-house orthogonal mesh generator for a Cartesian grid.

### List of Nodes
We first create a list of nodes for each dimension (X, Y, and Z). Here, all dimensions share the same node values.

The nodes will be spread from 0 to +2.

## List of Radii
We also create a list of radii for the centerpoint of each cell.

In [6]:
nodes = []
n_cells = 40
length = 1.0
xmin = 0.
dx = length / n_cells
for i in range(n_cells + 1):
    nodes.append(xmin + i * dx)

### Orthogonal Mesh Generation
We use the `OrthogonalMeshGenerator` and pass the list of nodes per dimension. Here, we pass 3 times the same list of
nodes to create a 3D geometry with square cells. Thus, we create a square domain, of side length 2, with a vertex on the origin (0,0), in the positive-positive-positive quadrant.

We also partition the 3D mesh into $2 \times 2$ subdomains using `KBAGraphPartitioner`. Since we want the split the x-axis in 2,
we give only 1 value in the xcuts array ($x=0$). Likewise for ycuts ($y=0$) and zcuts ($z=0$). The assignment to a partition is done based on where the
cell center is located with respect to the various xcuts, ycuts, and zcuts (in the code, a fuzzy logic is applied to avoid arithmetic issues).

In [7]:
meshgen = OrthogonalMeshGenerator(
    node_sets=[nodes, nodes, nodes],
    partitioner=KBAGraphPartitioner(
        nx=2,
        ny=2,
        nz=1,
        xcuts=[0.0],
        ycuts=[0.0]
    )
)
grid = meshgen.Execute()

[0]  Done checking cell-center-to-face orientations
[0]  00:00:03.3 Establishing cell connectivity.
[0]  00:00:03.3 Vertex cell subscriptions complete.
[0]  00:00:03.3 Surpassing cell 6400 of 64000 (10%)
[0]  00:00:03.3 Surpassing cell 12800 of 64000 (20%)
[0]  00:00:03.3 Surpassing cell 19201 of 64000 (30%)
[0]  00:00:03.3 Surpassing cell 25600 of 64000 (40%)
[0]  00:00:03.3 Surpassing cell 32000 of 64000 (50%)
[0]  00:00:03.4 Surpassing cell 38401 of 64000 (60%)
[0]  00:00:03.4 Surpassing cell 44801 of 64000 (70%)
[0]  00:00:03.4 Surpassing cell 51200 of 64000 (80%)
[0]  00:00:03.4 Surpassing cell 57600 of 64000 (90%)
[0]  00:00:03.4 Surpassing cell 64000 of 64000 (100%)
[0]  00:00:03.4 Establishing cell boundary connectivity.
[0]  00:00:03.4 Done establishing cell connectivity.
[0]  *** WARNING ***  KBAGraphPartitioner::Partition nx_*ny_*nz_ != number_of_parts
[0]  Number of cells per partition (max,min,avg) = 64000,64000,64000
[0]  
[0]  Mesh statistics:
[0]    Global cell count   

### Material IDs
When using the in-house `OrthogonalMeshGenerator`, no material IDs are assigned. The user needs to
assign material IDs to all cells. Here, we have a homogeneous domain, so we assign a material ID
with value 0 for each cell in the spatial domain.

In [8]:
grid.SetUniformBlockID(0)

[0]  00:00:05.3 Done setting block id 0 to all cells


In [9]:
def mat_id_function(pt, cur_id):
    if 0. < ((pt.x ** 2 + pt.y ** 2 + pt.z ** 2) ** (1/2)) < 0.5:
        return 1
    return cur_id

In [10]:
grid.SetBlockIDFromFunction(mat_id_function)

## Cross Sections
We create one-group cross sections using a built-in method. 
See the tutorials' section on cross sections for more details on how to load cross sections into OpenSn.

In [11]:
xs_mat = MultiGroupXS()
xs_mat.CreateSimpleOneGroup(sigma_t=1.,c=0.0)
xs_void = MultiGroupXS()
xs_void.CreateSimpleOneGroup(sigma_t=0.,c=0.0)

bsrc = [1.]

[0]  *** WARNING ***  Negative transport cross section found for group 0 in call to ComputeDiffusionParameters. sigma_t=0 sigma_1=0. Setting sigma_1=0, sigma_tr=sigma_t for this group.


In [27]:
D = [0.3333, 10000.0]
Q = [0.0, 0.0]
XSa = [1.0, 0.0]

def D_coef(i, pt):
    return D[i]

def Q_ext(i, pt):
    return Q[i]

def Sigma_a(i, pt):
    return XSa[i]

In [28]:
d_coef_fn = ScalarSpatialMaterialFunction(D_coef)
Q_ext_fn = ScalarSpatialMaterialFunction(Q_ext)
Sigma_a_fn = ScalarSpatialMaterialFunction(Sigma_a)

In [14]:
sphere_vol = SphereLogicalVolume()

sphere_bndry = "0"

grid.SetBoundaryIDFromLogicalVolume(sphere_vol, sphere_bndry, True)

## Volumetric Source
We create a volumetric multigroup source which will be assigned to cells with given block IDs.
Volumetric sources are assigned to the solver via the `options` parameter in the LBS block (see below).

## Angular Quadrature
We create a product Gauss-Legendre-Chebyshev angular quadrature and pass the total number of polar cosines
(here `npolar = 4`) and the number of azimuthal subdivisions in **four quadrants** (`nazimu = 4`).
This creates a 2D angular quadrature for XY geometry.

In [15]:
nazimu = 4
npolar = 2
pquad = GLCProductQuadrature3DXYZ(npolar, nazimu)
'''
{"boundary":sphere_bndry, "type": "dirichlet", "coeffs": [1.]},
'''

'\n{"boundary":sphere_bndry, "type": "dirichlet", "coeffs": [1.]},\n'

## Linear Boltzmann Solver
### Options for the Linear Boltzmann Problem (LBS)
In the LBS block, we provide
+ the number of energy groups,
+ the groupsets (with 0-indexing), the handle for the angular quadrature, the angle aggregation, the solver type,
tolerances, and other solver options.

In [29]:
phys = DFEMDiffusionSolver(
    name="problem_one",
    mesh=grid,
)
phys.SetOptions(boundary_conditions=[
    {"boundary": sphere_bndry, "type": "dirichlet", "coeffs": [1.0]}
])
phys.SetDCoefFunction(d_coef_fn)
phys.SetQExtFunction(Q_ext_fn)
phys.SetSigmaAFunction(Sigma_a_fn)

[0]  Boundary 0 set as Dirichlet with value 1


In [30]:
phys.Initialize()
phys.Execute()

[0]  
[0]  00:09:58.3 problem_one: Initializing DFEM Diffusion solver 
[0]  Global num cells: 64000
[0]  Boundary 0 set to dirichlet.
[0]  Num local DOFs: 512000
[0]  Num globl DOFs: 512000
[0]  
[0]  Executing DFEM IP Diffusion solver
[0]  Assembling system: 
[0]  Global assembly
[0]  Done global assembly
[0]  Solving: 
[0]  problem_one iteration    0 - Residual 6.5308517e-04
[0]  problem_one iteration    1 - Residual 1.2968540e-04
[0]  problem_one iteration    2 - Residual 1.0369915e-04
[0]  problem_one iteration    3 - Residual 3.4725570e-05
[0]  problem_one iteration    4 - Residual 2.3249765e-05
[0]  problem_one iteration    5 - Residual 1.8575311e-05
[0]  problem_one iteration    6 - Residual 8.1690202e-06
[0]  problem_one iteration    7 - Residual 6.7005292e-06
[0]  problem_one iteration    8 - Residual 5.2833263e-06
[0]  problem_one iteration    9 - Residual 2.6008136e-06
[0]  problem_one iteration   10 - Residual 2.1172809e-06
[0]  problem_one iteration   11 - Residual 1.32141

In [31]:
fflist = phys.GetFieldFunctions()
vtk_basename = "problem_one"
FieldFunctionGridBased.ExportMultipleToVTK(
    [fflist[0]],  # export only the flux of group 0 (first []), moment 0 (second [])
    vtk_basename
)

[0]  Exporting field functions to VTK with file base "problem_one"
[0]  Done exporting field functions to VTK.


In [32]:
def average_vol(vol0, r1, r2):
    ffvol = FieldFunctionInterpolationVolume()
    ffvol.SetOperationType("avg")
    ffvol.SetLogicalVolume(vol0)
    ffvol.AddFieldFunction(fflist[0])
    ffvol.Initialize()
    ffvol.Execute()
    avgval = ffvol.GetValue()
    print("Radius: {:.2f} {:.2f} {:.6f}".format(r1, r2, avgval))
    return avgval

def create_vols(N_vols, rmax):
    r_vals = np.linspace(0, rmax, N_vols + 1)
    vols = np.empty(N_vols)
    avgphi = np.zeros(N_vols)
    for i in range(N_vols):
        if i != 0:
            inner_vol = SphereLogicalVolume(r=r_vals[i])
            outer_vol = SphereLogicalVolume(r=r_vals[i + 1])
            vol = BooleanLogicalVolume(parts=[{"op":True,"lv":outer_vol},{"op":False,"lv":inner_vol}])
        else:
            vol = SphereLogicalVolume(r=r_vals[i + 1])
        avgphi[i] = average_vol(vol, r_vals[i], r_vals[i+1])
    return avgphi

In [33]:
n_vols = 10
sim_vals = create_vols(n_vols, 1.)

Radius: 0.00 0.10 0.999994
Radius: 0.10 0.20 1.000007
Radius: 0.20 0.30 0.999955
Radius: 0.30 0.40 0.999965
Radius: 0.40 0.50 0.999979
Radius: 0.50 0.60 0.950938
Radius: 0.60 0.70 0.856737
Radius: 0.70 0.80 0.760009
Radius: 0.80 0.90 0.638615
Radius: 0.90 1.00 0.447585


In [34]:
def E2(x):
    return np.exp(-x)-x*exp1(x)

def get_phi(r, psi, a, sig):
    phi = (psi/(2*r))*((np.exp(-sig*(a-r))-np.exp(-sig*(a+r)))/sig + (r+a)*exp2(sig*(a-r)) - (a-r)*exp2(sig*(a+r)))
    return phi        

In [39]:
psi = 1.
a = 1.
sig = 1.
r_vals = np.linspace(0.1, 1., n_vols)
phi = np.zeros(n_vols)
for i in range(n_vols):
    phi[i] = get_phi(r_vals[i], psi, a, sig)
    print(r_vals[i], phi[i])

0.1 0.9858946242908428
0.2 0.9988468403011536
0.30000000000000004 1.0205126043345438
0.4 1.051010465424381
0.5 1.0905072807507512
0.6 1.139219110057747
0.7000000000000001 1.1974123740627562
0.8 1.2654052832588492
0.9 1.3435695450097547
1.0 1.4323323583816936


In [38]:
err = np.zeros(n_vols)
err[:] = abs(phi[:]-sim_vals[:])/phi[:]
print(err)

[0.01430129 0.00116143 0.02014416 0.04856815 0.08301526 0.16527196
 0.28450965 0.39939474 0.52468751 0.68751319]


## Finalize (for Jupyter Notebook only)

In Python script mode, PyOpenSn automatically handles environment termination. However, this
automatic finalization does not occur when running in a Jupyter notebook, so explicit finalization
of the environment at the end of the notebook is required. Do not call the finalization in Python
script mode, or in console mode.

Note that PyOpenSn's finalization must be called before MPI's finalization.


In [40]:
from IPython import get_ipython

def finalize_env():
    Finalize()
    MPI.Finalize()

ipython_instance = get_ipython()
if ipython_instance is not None:
    ipython_instance.events.register("post_execute", finalize_env)


Elapsed execution time: 00:21:25.8
2025-05-10 11:58:30 OpenSn finished execution.


## Possible Extensions
1. Change the number of MPI processes;
2. Change the spatial resolution by increasing or decreasing the number of cells;
3. Change the angular resolution by increasing or decreasing the number of polar and azimuthal subdivisions.